In [45]:
import statistics as stats
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import feedparser

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC   # acciones encadenadas
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options



import tqdm
from bs4 import BeautifulSoup
import time
import re
import json

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')

from pymongo import MongoClient

In [46]:
options=Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-allow-origins=*")
options.add_argument("user-data-dir=cookies")
#options.add_argument('--headless')                 #Habilitar si no queremos ver la ventana
options.add_experimental_option("detach", True)    #Esta opción corrige el error de cierre repentino
options.add_argument('--start-minimized')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')

In [32]:
driver = webdriver.Chrome(options=options)    # abre una ventana de chrome 

#### 1. Empezamos por importar el csv que necesito y detallar la informacion que me interesa.

 - Voy a llamar a mi dataframe para llevar a cabo un diccionario en el que llame y se la ubicacion de cada uno de los locales.
 
- Posteriormente llevo a cabo con geopy la localizacion de los locales. 

In [24]:
df_test = pd.read_csv('../datos/data_rest.csv')

In [25]:
df_test.head()

,restaurante,gastronomia,precio,opinion,reseña
0,Yakiniku Rikyu,Japonesa,Medio,267 opiniones,Patrocinado
1,marmitón,Mediterránea,Medio,89 opiniones,MICHELIN
2,Vinoteca Moratín,Internacional,Medio,1221 opiniones,MICHELIN
3,Pilar Akaneya,Japonesa,Alto,276 opiniones,MICHELIN
4,Gioia,Italiana,Medio,387 opiniones,MICHELIN


In [26]:
len(df_test["restaurante"].unique())

295

In [27]:
duplicados = df_test["restaurante"].duplicated().sum()
duplicados

5

In [28]:
df_test.drop_duplicates(subset='restaurante', inplace=True)

In [29]:
df_test["restaurante"].unique()

array(['Yakiniku Rikyu', 'marmitón', 'Vinoteca Moratín', 'Pilar Akaneya',
       'Gioia', 'Noi Ristorante', 'Mudrá Plant Based Recoletos',
       'La MaMá Restaurante', 'Sisapo', 'RECREO Espartinas', 'Miyama',
       'Deessa', 'Hotaru Madrid', 'Yugo the Bunker', 'Coque',
       'El pedrusco de Aldealcorvo', 'KABUKI WELLINGTON SL', 'Kabuki',
       'Restaurante Lakasa', 'Bardero', 'Pizzart Fuencarral',
       'Ornella Velázquez', 'Lettera Trattoria moderna',
       'Pizzart Villa Canalejas', 'Pummarola', 'Brunchit Malasaña',
       'Ouh...Babbo!', 'Restaurante Excalibur Madrid', 'Rigatoni',
       'Davanti Food & Market', 'Entre Santos Madrid',
       'Trattoria Malatesta', 'Minelli 1960',
       'Oven Mozzarella Gran Vía 6', "Lab '84",
       'Sibuya Urban Sushi Bar Madrid C/Orense', 'Gustazio Gastrobar',
       'Smok Mok', 'Piccola Napoli', 'Da Luca', 'Il Pizzaiolo',
       'Pastamore', 'Oven Mozzarella', 'Gourmet Experience Gran Vía',
       'Proverbium', 'Dans Le Noir ?', 'Kamado As



for i in df_test["restaurante"].unique():
    funcion_de_escrapeo_para_google_(i)

#### 1. Empezamos el scrapeo

- Creo una función con la que se me genere una lista, con los nombres de los restaurantes y la ubicacion de estos. 
- Esta ubicación se busca a través del scrapeo en google de la ubicacion de los restaurantes que le digo.

In [40]:
def obtener_ubicacion_restaurante(restaurante):
    ubicacion_dict = {}

    try:
        # Realizar búsqueda en Google y obtener el primer resultado
        query = f"ubicación {restaurante}"
        url = f"https://www.google.com/search?q={query}"

        # Abrir la página en el navegador
        driver.get(url)

        # Esperar un breve momento para que la página cargue (puedes ajustar esto según sea necesario)
        time.sleep(3)

        # Obtener el contenido de la página después de que ha cargado
        page_content = driver.page_source

        # Analizar el contenido con Beautiful Soup
        soup = BeautifulSoup(page_content, 'html.parser')

        # Extraer la ubicación si está presente
        ubicacion_element = soup.find('div', class_='sXLaOe')
        ubicacion = ubicacion_element.get_text(strip=True) if ubicacion_element else None

        # Almacenar en el diccionario
        ubicacion_dict[restaurante] = ubicacion

    except:
        print(f"La página para {restaurante} no ha funcionado correctamente.")
        

    return ubicacion_dict


In [41]:
for restaurante in df_test["restaurante"].unique():
    resultado = obtener_ubicacion_restaurante(restaurante)
    if restaurante in resultado:
        print(f"Restaurante: {restaurante}, Ubicación: {resultado[restaurante]}")
    else:
        print(f"Restaurante: {restaurante}, No se encontró ubicación")

Restaurante: Yakiniku Rikyu, Ubicación: P.º de la Castellana, 15, 28046 Madrid
Restaurante: marmitón, Ubicación: C. de las Aguas, 6, 28005 Madrid
Restaurante: Vinoteca Moratín, Ubicación: C. de Moratín, 36, 28014 Madrid
Restaurante: Pilar Akaneya, Ubicación: C. de Espronceda, 33, 28003 Madrid
Restaurante: Gioia, Ubicación: C. de San Bartolomé, 23, 28004 Madrid
Restaurante: Noi Ristorante, Ubicación: C. de Recoletos, 6, 28001 Madrid
Restaurante: Mudrá Plant Based Recoletos, Ubicación: C. de Recoletos, 13, 28001 Madrid
Restaurante: La MaMá Restaurante, Ubicación: 6 bis, Parque, Av. de Brasil, 28020 Madrid
Restaurante: Sisapo, Ubicación: C. de Trafalgar, 14, 28010 Madrid
Restaurante: RECREO Espartinas, Ubicación: C. de Espartinas, Nº 5, 28001 Madrid
Restaurante: Miyama, Ubicación: None
Restaurante: Deessa, Ubicación: Pl. de la Lealtad, 5, 28014 Madrid
Restaurante: Hotaru Madrid, Ubicación: C. de Alcalá, 99, 28009 Madrid
Restaurante: Yugo the Bunker, Ubicación: Calle de San Blas, 4, 28014 